# Simple component separation

In [152]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pymaster as nmt
import healpy as hp
import pysm3

from fgbuster import (CMB, Dust, Dustmom, Synchrotron, 
                      basic_comp_sep, 
                      get_observation, get_instrument)
from fgbuster.visualization import corner_norm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Simulate your frequency maps
You have some frequecy maps to clean, they can be either data or simulations. For example, you can use `pysm` to simulate microwave skys with different complexities. ForeGroundBuster adds a couple of functions that make the process even easier.

In [153]:
nside = 32
instrument = get_instrument('LiteBIRD')
sky = pysm3.Sky(nside=nside, preset_strings=['d1','s0','c1'])
freq_maps = get_observation(instrument, sky, nside=nside)
fsky=0.7
cut = {0.5:53,0.6:80,0.7:121}
mask0 = hp.read_map(Pr+"Moments/masks/mask_nside512_fsky%spc_P353_smooth10deg_cut%smuK.fits"%(int(fsky*100),cut[fsky]))
mask0 = hp.ud_grade(mask0,nside_out=nside)
mask = nmt.mask_apodization(mask0, 10, apotype='C2')
b = nmt.bins.NmtBin(nside=nside,lmax=2*nside+1,nlb=5)
ell = b.get_effective_ells()

#mask=mask0
freq_maps = freq_maps*mask


## Define what you fit for
Create your sky model as a list of components

In [154]:
components = [CMB(), Dust(353.), Synchrotron(20.)]

## Component separation
The tools inside ForeGroundBuster allow for very flexible and diverse component separation approaches. However, we also provide a set of predefined function that perform component separation out of the box. They suit most common use cases.

In [155]:
result = basic_comp_sep(components, instrument, freq_maps)

## Explore the results
You have just solved for both the spectral parameters of your components and their amplitudes.

Get the spectral parameters name and values with

In [156]:
print(result.params)
print(result.x)

['Dust.beta_d', 'Dust.temp', 'Synchrotron.beta_pl']
[ 1.51496242 20.87164808 -2.49908406]


You can also have a look at their semi-analytic covariance, but remember that it is accurate only the high signal-to-noise regime

In [ ]:
components = [CMB(), Dustmom(353.,beta_d=result.x[0],temp=result.x[1],border=2,torder=0), Synchrotron(20.)]
resultmom = basic_comp_sep(components, instrument, freq_maps)

componentsT = [CMB(), Dustmom(353.,beta_d=result.x[0],temp=result.x[1],border=0,torder=2), Synchrotron(20.)]
resultmomt = basic_comp_sep(componentsT, instrument, freq_maps)

## Explore the results
You have just solved for both the spectral parameters of your components and their amplitudes.

Get the spectral parameters name and values with

In [ ]:
print(resultmom.params)
print(resultmom.x)

In [ ]:
corner_norm(resultmom.x, resultmom.Sigma, labels=resultmom.params)

The amplitudes of the components are stacked in the ``s`` attribute and they are in the same format of the input frequency maps: Q and U healpix maps, in this case.

In [ ]:
def compute_master(f_a, f_b, wsp):
    cl_coupled = nmt.compute_coupled_cell(f_a, f_b)
    cl_decoupled = wsp.decouple_cell(cl_coupled)
    return cl_decoupled

incmb = sky.components[2].map[:].value

def getres(result):
    cmb = np.array([np.ones((result.s[0, 0].shape)),result.s[0, 1],result.s[0, 2]])
    fa1 = nmt.NmtField(mask,(cmb[1:]-incmb[1:])*mask,purify_e=True, purify_b=True)
    wsp = nmt.NmtWorkspace()
    wsp.compute_coupling_matrix(fa1, fa1, b)
    cl = compute_master(fa1,fa1,wsp)        
    return cl[3]


clBB = getres(result)       
clBBmom= getres(resultmom)
clBBmomt= getres(resultmomt)

DL_tens = b.bin_cell(np.load(Pr+"/Moments/CLsimus/DLtensor_CAMBparamPlanck2018_r=1.npy")[2,0:2*nside+2])

In [ ]:
plt.plot(ell,clBB,label='mbb')
plt.plot(ell,clBBmom,label='+momt')
plt.plot(ell,clBBmomt,label='+momb')
plt.plot(ell,1e-3*2*np.pi*DL_tens/(ell*(ell+1)),label='BB,r=1e-3',c='k',linestyle='--')
plt.legend()
plt.loglog()

In [ ]:
hp.mollview((cmb[1]-incmb[1])*mask)